In [1]:
import os
os.chdir ("D:/TensorFlow")

In [3]:
import tensorflow as tf

In [2]:
# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


# Setting hyperparameters

In [4]:
#if learning rate is to big it skip the optimal solution and too small too many iterations
learning_rate = 0.01 # A known decent learning rate
training_iteration = 30
batch_size = 100
display_step = 2

In [5]:
# TF graph input
## intializing placeholder with no dataset and will assigned on later stage
x = tf.placeholder("float", [None, 784]) # mnist data image of shape 28*28=784
y = tf.placeholder("float", [None, 10]) # 0-9 digits recognition => 10 classes


# Create a model

In [7]:
# Set model weights
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

In [8]:
# creating name scope which help organize nodes in graph later used in tensorboard
with tf.name_scope("Wx_b") as scope:
    # Construct a linear model ---- implying logistic regression where input is 'x' and weight is 'W' + 'b' Bias
    model = tf.nn.softmax(tf.matmul(x, W) + b) # Softmax

In [9]:
# Add summary operations to collect data, and visaulize our weights and biases
w_h = tf.summary.histogram("weights", W)
b_h = tf.summary.histogram("biases", b)

## Creating three scope to visaulize performance on tensorboard

In [10]:
# More name scopes will clean up graph representation
## specifying cost function to minimize error during training
with tf.name_scope("cost_function") as scope:
    # Minimize error using cross entropy function
    # Cross entropy
    cost_function = -tf.reduce_sum(y*tf.log(model))
    # Create a scalar summary to monitor the cost function later
    tf.summary.scalar("cost_function", cost_function)
# it will create optimize function to help minimizing error 
with tf.name_scope("train") as scope:
    # Gradient descent take the following two as parameter
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost_function)

In [13]:
# Initializing the variables
init = tf.global_variables_initializer()

In [14]:
# Merge all summaries into a single operator
merged_summary_op = tf.summary.merge_all()

# Launch the graph

In [20]:
## Initializing session that execute dataflow graph
with tf.Session() as sess:
    sess.run(init)
    
    # Change this to a location on your computer
    summary_writer = tf.summary.FileWriter('/TensorFlow/', graph_def=sess.graph_def)

   
    # Training cycle
    for iteration in range(training_iteration):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            # Fit training using batch data
            sess.run(optimizer, feed_dict={x: batch_xs, y: batch_ys})
            # Compute the average loss
            avg_cost += sess.run(cost_function, feed_dict={x: batch_xs, y: batch_ys})/total_batch
            # Write logs for each iteration
            summary_str = sess.run(merged_summary_op, feed_dict={x: batch_xs, y: batch_ys})
            summary_writer.add_summary(summary_str, iteration*total_batch + i)
        # Display logs per iteration step
        if iteration % display_step == 0:
            print ("Iteration:", '%04d' % (iteration + 1), "cost=", "{:.9f}".format(avg_cost))

    print ("Tuning completed!")
     # Test the model
    predictions = tf.equal(tf.argmax(model, 1), tf.argmax(y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(predictions, "float"))
    print ("Accuracy:", accuracy.eval({x: mnist.test.images, y: mnist.test.labels}))

Iteration: 0001 cost= 29.678052787
Iteration: 0003 cost= 20.995571470
Iteration: 0005 cost= 20.131782292
Iteration: 0007 cost= 19.717234615
Iteration: 0009 cost= 19.468563095
Iteration: 0011 cost= 19.089636438
Iteration: 0013 cost= 18.906066835
Iteration: 0015 cost= 18.788357228
Iteration: 0017 cost= 18.627161444
Iteration: 0019 cost= 18.490115288
Iteration: 0021 cost= 18.439471263
Iteration: 0023 cost= 18.285395820
Iteration: 0025 cost= 18.271101155
Iteration: 0027 cost= 18.261510277
Iteration: 0029 cost= 18.110410109
Tuning completed!
Accuracy: 0.9228
